In [1]:
using JuMP, Gurobi, CSV, LinearAlgebra, DataFrames, Random, Distributions, Statistics

In [1]:
using Random, CategoricalArrays, DataFrames, DataFramesMeta, StatsBase, CSV
println("##########################################################################################")
println("##########################################################################################")
println("Starting OptImpute for blunt injuries only")
println("##########################################################################################")
println("##########################################################################################")
#data_path = "../Data/time_split_per_injury_new_morbidity/blunt/"
data_path = "../Data/time_split_per_injury_new_morbidity/penetrating/"
train_X_time = CSV.read(data_path*"trauma_X_train_time_penetrating.csv")
test_X_time = CSV.read(data_path*"trauma_X_test_time_penetrating.csv")
println("Loaded X train and test data with time split for blunt")

# Dropped tmode1 and hemorrhage_ctrl_type and severity_max (29/11/2019)
names_with_alcohol = [
    :age, :alcohol,:gender, :race1, :acslevel, #:tmode1,
    :signsoflife, :sbp1, :pulse1, :oxysat1, :temp1, :gcstot1, :bleeding_disorder,
    :current_chemotherapy, :congestive_heart_failure, :current_smoker,
    :chronic_renal_failure, :history_cva, :diabetes, :disseminated_cancer,
    :copd, :steroid, :cirrhosis, :history_MI, :history_pvd, :hypertension_medication,
    :method_of_injury, :Face_severity, :Neck_severity, :Thorax_severity, :Abdomen_severity,
    :Spine_severity, :Upper_Extremity_severity, :Lower_Extremity_severity, :Pelvis_Perineum_severity, :External_severity,
    #:severity_max, #:hemorrhage_ctrl_type
]
train_X_time = train_X_time[!, names_with_alcohol]
test_X_time = test_X_time[!, names_with_alcohol]
println("Kept only relevant columns")

severity_categorical_var = [
    :Face_severity,
    :Neck_severity,
    :Thorax_severity,
    :Abdomen_severity,
    :Spine_severity,
    :Upper_Extremity_severity,
    :Lower_Extremity_severity,
    :Pelvis_Perineum_severity,
    #:severity_max
]
#for col_name in severity_categorical_var
#    train_X_time[!, col_name] = CategoricalArray(train_X_time[!, col_name], ordered=true)
#    test_X_time[!, col_name] = CategoricalArray(test_X_time[!, col_name], ordered=true)
#    println("Transformed column '", col_name, "' to Ordered Categorical")
#end
#categorical!(train_X_time)
#categorical!(test_X_time)
#println("Transformed all other non-ordered Categorical Variables")

k_neighbors_blunt = 200

##########################################################################################
##########################################################################################
Starting OptImpute for blunt injuries only
##########################################################################################
##########################################################################################
Loaded X train and test data with time split for blunt
Kept only relevant columns


200

In [21]:
data_path = "../Data/time_split_per_injury_new_morbidity/penetrating/"
train_X_time = CSV.read(data_path*"trauma_X_train_time_penetrating.csv")
test_X_time = CSV.read(data_path*"trauma_X_test_time_penetrating.csv")

train_y_mortality_time = CSV.read(
    data_path*"trauma_y_train_mortality_time_penetrating.csv",
    header=false
)
test_y_mortality_time = CSV.read(
    data_path*"trauma_y_test_mortality_time_penetrating.csv",
    header=false
)
train_y_mortality_time = convert(Matrix, train_y_mortality_time)[:,1]
test_y_mortality_time = convert(Matrix, test_y_mortality_time)[:,1]

train_y_morbidity_time = CSV.read(
    data_path*"trauma_y_train_morbidity_time_penetrating.csv",
    header=false
)
test_y_morbidity_time = CSV.read(
    data_path*"trauma_y_test_morbidity_time_penetrating.csv",
    header=false
)
train_y_morbidity_time = convert(Matrix, train_y_morbidity_time)[:,1]
test_y_morbidity_time = convert(Matrix, test_y_morbidity_time)[:,1]
println("Loaded all data with time split")

Loaded all data with time split


In [22]:
# Dropped tmode1 and hemorrhage_ctrl_type and severity_max (29/11/2019)
names_with_alcohol = [
    :age, :alcohol,:gender, :race1, :acslevel, #:tmode1,
    :signsoflife, :sbp1, :pulse1, :oxysat1, :temp1, :gcstot1, :bleeding_disorder,
    :current_chemotherapy, :congestive_heart_failure, :current_smoker,
    :chronic_renal_failure, :history_cva, :diabetes, :disseminated_cancer,
    :copd, :steroid, :cirrhosis, :history_MI, :history_pvd, :hypertension_medication,
    :method_of_injury, :Face_severity, :Neck_severity, :Thorax_severity, :Abdomen_severity,
    :Spine_severity, :Upper_Extremity_severity, :Lower_Extremity_severity, 
    :Pelvis_Perineum_severity, :External_severity,
    #:severity_max, #:hemorrhage_ctrl_type
]
train_X_time = train_X_time[!, names_with_alcohol]
test_X_time = test_X_time[!, names_with_alcohol]
println("Kept only relevant columns")

severity_categorical_var = [
    :Face_severity,
    :Neck_severity,
    :Thorax_severity,
    :Abdomen_severity,
    :Spine_severity,
    :Upper_Extremity_severity,
    :Lower_Extremity_severity,
    :Pelvis_Perineum_severity,
    #:severity_max
]
#for col_name in severity_categorical_var
#    train_X_time[!, col_name] = CategoricalArray(train_X_time[!, col_name], ordered=true)
#    test_X_time[!, col_name] = CategoricalArray(test_X_time[!, col_name], ordered=true)
#    println("Transformed column '", col_name, "' to Ordered Categorical")
#end
#categorical!(train_X_time)
#categorical!(test_X_time)
#println("Transformed all other non-ordered Categorical Variables")

Kept only relevant columns


8-element Array{Symbol,1}:
 :Face_severity           
 :Neck_severity           
 :Thorax_severity         
 :Abdomen_severity        
 :Spine_severity          
 :Upper_Extremity_severity
 :Lower_Extremity_severity
 :Pelvis_Perineum_severity

In [25]:
train_X_time_subset = train_X_time[1:400, :]

,age,alcohol,gender,race1,acslevel,signsoflife,sbp1,pulse1,oxysat1
,Float64⍰,Int64,Float64⍰,Int64,Int64,Int64,Float64⍰,Float64⍰,Float64⍰
1,23.0,1,1.0,0,1,-1,145.0,131.0,missing
2,47.0,0,0.0,1,-1,-1,174.0,95.0,100.0
3,41.0,0,0.0,1,1,-1,144.0,124.0,100.0
4,28.0,1,0.0,1,2,-1,114.0,98.0,missing
5,42.0,0,0.0,1,1,-1,181.0,106.0,99.0
6,56.0,-1,0.0,1,2,-1,0.0,0.0,missing
7,20.0,-1,0.0,2,1,-1,110.0,92.0,missing
8,18.0,-1,0.0,0,2,-1,59.0,108.0,92.0
9,16.0,-1,0.0,1,2,-1,160.0,94.0,99.0


In [26]:
k_neighbors_penetrating = 100
# Fitting OptImpute on train set
lnr_optimpute_train = IAI.ImputationLearner(method=:opt_knn, knn_k=k_neighbors_penetrating)
IAI.fit!(lnr_optimpute_train, train_X_time)
train_X_time = IAI.transform(lnr_optimpute_train, train_X_time)

,age,alcohol,gender,race1,acslevel,signsoflife,sbp1,pulse1
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,23.0,1.0,1.0,0.0,1.0,-1.0,145.0,131.0
2,47.0,0.0,0.0,1.0,-1.0,-1.0,174.0,95.0
3,41.0,0.0,0.0,1.0,1.0,-1.0,144.0,124.0
4,28.0,1.0,0.0,1.0,2.0,-1.0,114.0,98.0
5,42.0,0.0,0.0,1.0,1.0,-1.0,181.0,106.0
6,56.0,-1.0,0.0,1.0,2.0,-1.0,0.0,0.0
7,20.0,-1.0,0.0,2.0,1.0,-1.0,110.0,92.0
8,18.0,-1.0,0.0,0.0,2.0,-1.0,59.0,108.0
9,16.0,-1.0,0.0,1.0,2.0,-1.0,160.0,94.0


In [29]:
CSV.write("./train_X_time_subset.csv",train_X_time_subset)

"./train_X_time_subset.csv"

In [ ]:
# Fitting OptImpute on test set:
lnr_optimpute_test = IAI.ImputationLearner(method=:opt_knn, knn_k=k_neighbors_penetrating, cluster=false)
IAI.fit!(lnr_optimpute_test, test_X_time)
test_X_time = IAI.transform(lnr_optimpute_test, test_X_time)

In [ ]:
lnr_optimpute_test = IAI.ImputationLearner(method=:opt_knn, knn_k=k_neighbors_penetrating)
IAI.fit!(lnr_optimpute_test, test_X_time)
test_X_time = IAI.transform(lnr_optimpute_test, test_X_time)